In [ ]:
import sys
sys.path.insert(0, '../driver')

In [ ]:
import numpy as np
import ephem
import plotly.graph_objects as go
from pyquaternion import Quaternion
from control import PID_Controller, polar_rotation_angle, MotorSpeedController, CalibrationManager
from shr import hms2rad, rad2deg, dms2rad, rad2hms, deg2dms, rad2dms
import plotly.graph_objects as go
import asyncio
import logging


def rms(x):
    return rad2dms(x).replace('d','°')

def dms(x):
    return deg2dms(x).replace('d','°')

In [ ]:
cm = CalibrationManager()
motors = [ MotorSpeedController(None, cm, axis, None) for axis in range(3) ]
logger = logging.getLogger()  
pid = PID_Controller(logger, motors)

n_steps = 300
pid.measure(np.array([45, 5, 0], dtype=float), np.array([45, 5, 0], dtype=float))
pid.set_alpha_target({ "az":90, "alt":20, "roll":0 })
pid.is_moving = True
pid.dt = 0.2

# Logging arrays
theta_log = []
ref_log = []
ctrl_log = []
move_log = []

# Simulation loop
for i in range(n_steps):
    pid.predict()
    if i==100:
        pid.set_alpha_axis_velocity(2, .005)
    if i==150:
        pid.set_alpha_axis_velocity(2, 0)
    await pid.control_step()
    theta_log.append(pid.theta_meas.copy())
    ref_log.append(pid.theta_ref.copy())
    ctrl_log.append(pid.omega_op.copy())
    move_log.append(pid.is_moving)

# Convert logs to arrays
theta_log = np.array(theta_log)
ref_log = np.array(ref_log)
ctrl_log = np.array(ctrl_log)
move_log = np.array(move_log)

### 📊 Plotly Visualization: θ₁, θ₂, θ₃
motor_labels = ['θ₁', 'θ₂', 'θ₃']
figs = []
time = np.linspace(0, pid.dt * n_steps, n_steps)

threshold_deg_squared = (10 / 60) ** 2  # 10 arcmin squared in degrees²

# Find intervals where cost exceeds threshold
transitions = np.diff(move_log.astype(int))
# Start and end indices of each high-cost region
start_idxs = np.where(transitions == 1)[0] + 1
end_idxs = np.where(transitions == -1)[0] + 1
# Handle edge case: region starts from beginning or goes to end
if move_log[0]:
    start_idxs = np.insert(start_idxs, 0, 0)
if move_log[-1]:
    end_idxs = np.append(end_idxs, len(move_log))

# Plot each motor
for i in range(3):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=ref_log[:, i], name='Reference', line=dict(color='gray', dash='dash')))
    fig.add_trace(go.Scatter(x=time, y=theta_log[:, i], name='Measured', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=time, y=ctrl_log[:, i], name='Control ω', line=dict(color='green')))

    # Add shaded shapes for each moving segment
    for start, end in zip(start_idxs, end_idxs):
        fig.add_shape( type="rect",
            x0=time[start], x1=time[end - 1],
            y0=min(theta_log[:, i]) - 5, y1=max(theta_log[:, i]) + 5,
            fillcolor="rgba(255, 0, 0, 0.15)", line=dict(width=0), layer="below"
        )

    fig.update_layout(
        title=f'Motor Angle {motor_labels[i]}',
        xaxis_title='Time (seconds)',
        yaxis_title='Degrees',
        template='plotly_white',
        showlegend=True
    )
    fig.show()



In [ ]:
import numpy as np
import plotly.graph_objects as go

# Simulated tiny step change sequence around zero
t = np.linspace(0, 5, 500)
step_change = 0.05 * np.sin(2 * np.pi * 1 * t)  # Small oscillation around zero

# Parameters
deadband_thresh = 0.02
hysteresis_thresh = 0.03
alpha = 0.2  # Low-pass filter coefficient

# Memory for control signal
omega_raw = step_change.copy()
omega_deadband = np.zeros_like(t)
omega_hysteresis = np.zeros_like(t)
omega_filtered = np.zeros_like(t)

prev_error = 0.0
prev_output = 0.0

for i in range(len(t)):
    error = omega_raw[i]

    # Apply deadband
    if np.abs(error) < deadband_thresh:
        omega_deadband[i] = 0.0
    else:
        omega_deadband[i] = error

    # Apply hysteresis
    if np.sign(error) != np.sign(prev_error) and np.abs(error) < hysteresis_thresh:
        omega_hysteresis[i] = 0.0
    else:
        omega_hysteresis[i] = omega_deadband[i]

    # Apply low-pass filter (first-order lag)
    omega_filtered[i] = alpha * omega_hysteresis[i] + (1 - alpha) * prev_output

    # Update history
    prev_error = error
    prev_output = omega_filtered[i]

# 📊 Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=omega_raw, mode='lines', name='Raw Signal', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=t, y=omega_deadband, mode='lines', name='Deadband Applied', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=t, y=omega_hysteresis, mode='lines', name='Hysteresis Applied', line=dict(color='green')))
fig.add_trace(go.Scatter(x=t, y=omega_filtered, mode='lines', name='Filtered Output', line=dict(color='blue')))

fig.update_layout(
    title='Control Signal Refinement to Suppress Backlash-Induced Reversals',
    xaxis_title='Time (s)',
    yaxis_title='Angular Velocity Command',
    template='plotly_white'
)

fig.show()

In [ ]:

# Set up observer
observer = ephem.Observer()
observer.lat = '-33.859887'
observer.lon = '151.202177'
observer.lat = '-30.4100413'
observer.lon = '152.6370429'
observer.elevation = 39
observer.date = ephem.now()  # Use current time for observer
observer.epoch = ephem.J2000

# 47 Tuc (J2000)
ra = '0h24m07.23s'
dec = '-72d04m35.0s'

ra0 = hms2rad(ra)
dec0 = dms2rad(dec)

# Assign precessed coordinates
body = ephem.FixedBody()
body._ra=ra0
body._dec=dec0
body._epoch=ephem.J2000  # Use J2000 epoch for RA/Dec
body.compute(observer)

az = rad2deg(body.az)
alt = rad2deg(body.alt) 
pa = polar_rotation_angle(observer.lat, body.az, body.alt)

print(f"Target:        RA = {rad2hms(ra0)},  Dec = {rms(dec0)}")
print(f"Alt/Az/Pra:    Az = {dms(az)},  Alt = {dms(alt)},  Polar Rotation Angle = {dms(pa)}")

In [ ]:
N = 60
Δt = 1

azaltroll_ref = compute_body_trajectory(N, Δt, observer, body, roll=15, is_equatorial_roll=False)
for row in azaltroll_ref:
    print([dms(x) for x in row])

In [ ]:
theta_ref = compute_desired_motor_angles(azaltroll_ref)
for row in theta_ref:
    print([dms(x) for x in row])